In [32]:
import pandas as pd

from src.constant import MAIN_DIR
from src.database import DB
from tqdm.auto import tqdm

PHASE2_DIR = MAIN_DIR / "archive" / "phase2"
N_TRAIN_LIST = ["25"]
POLICY_LIST = ["baseline", "ea", "eb", "ec", "ia", "ib"]

In [34]:
records = []
total_iterations = len(N_TRAIN_LIST) * len(POLICY_LIST)
progress_bar = tqdm(total=total_iterations, desc="Processing")

for n_train in N_TRAIN_LIST:
    for policy in POLICY_LIST:
        # Update description to show current n_train and policy
        progress_bar.set_description(f"n_train={n_train}, policy={policy}")
        
        db_path_list = list((PHASE2_DIR / n_train / policy).glob("run-policy-*.db"))

        for db_path in db_path_list:
            results = DB(db_path).get_results()
            cost = (
                results.loc[results["prefix"].str.startswith("test")]
                .groupby(["instance_id", "prefix"])["cost"]
                .min()  # min for every problem (of 2 solvers)
                .reset_index()
                .groupby("instance_id")["cost"]
                .median()  # median over 5 runs
                .mean()  # total mean score
            )
            cpu_time = (
                results.loc[
                    results["prefix"].str.startswith("config")
                    & results["cached"].eq(0)
                    & results["surrogate"].eq(0),
                    "time",
                ].sum()
                / 3600
            )
            records.append(
                {
                    "db_path": "/".join(db_path.parts[-5:]),
                    "n_train": n_train,
                    "policy": policy,
                    "cost": cost,
                    "cpu_time": cpu_time,
                }
            )        
        progress_bar.update(1)

progress_bar.close()

df = pd.DataFrame(records)
df

Processing:   0%|          | 0/6 [00:00<?, ?it/s]

,db_path,n_train,policy,cost,cpu_time
0,archive/phase2/25/baseline/run-policy-baseline...,25,baseline,0.20404,1.085407
1,archive/phase2/25/baseline/run-policy-baseline...,25,baseline,0.18832,1.231176
2,archive/phase2/25/baseline/run-policy-baseline...,25,baseline,0.73652,1.443054
3,archive/phase2/25/baseline/run-policy-baseline...,25,baseline,0.30916,0.832994
4,archive/phase2/25/baseline/run-policy-baseline...,25,baseline,0.20116,1.118927
5,archive/phase2/25/baseline/run-policy-baseline...,25,baseline,0.52012,0.832707
6,archive/phase2/25/ea/run-policy-ea-25-1012835.db,25,ea,0.63916,1.032151
7,archive/phase2/25/ea/run-policy-ea-25-1012836.db,25,ea,0.33880,0.644563
8,archive/phase2/25/ea/run-policy-ea-25-1012837.db,25,ea,0.28236,0.760477
9,archive/phase2/25/ea/run-policy-ea-25-1012838.db,25,ea,0.21496,0.920378


In [47]:
df.pivot_table(index="policy", columns="n_train", values="cpu_time", aggfunc="mean")

n_train,25
policy,
baseline,1.090711
ea,0.800620
eb,0.572137
ec,1.795801
ia,0.733699
ib,0.405797


In [48]:
df.pivot_table(index="policy", columns="n_train", values="cost", aggfunc="mean")

n_train,25
policy,
baseline,0.359887
ea,0.370954
eb,0.360829
ec,0.222030
ia,0.240851
ib,0.304771
